In [1]:
#import pandas as pd NO LO USAMOS MAS
import gzip
import polars as pl
import numpy as np


## Leer data

In [22]:
def read_polar(path):
    # Ruta al archivo — importante usar r"" para evitar problemas con backslashes en Windows
    #path = r"G:\My Drive\dmeyf\competencia_02_crudo.csv.gz"

    # Leer el CSV comprimido
    dataset = pl.read_csv(
        path,
        separator=",",            # usa ',' como separador
        has_header=True,    # primera fila son los nombres de columna
        infer_schema_length=5000,  # lee más filas para detectar bien los tipos
        ignore_errors=True, # salta filas corruptas si las hubiera
        low_memory=True     # usa menos RAM si el archivo es grande
    )

    #print(dataset.shape)
    #print(dataset.head())
    return dataset

def write_polar_gz(dataset,path_target):
    # === 7. Guardar resultado ===
    with gzip.open(path_target, "wt", encoding="utf-8") as f:
        dataset.write_csv(f)
    return

def generar_ternaria(dataset):
    # === 2. Calcular periodo0 consecutivo ===
    dataset = dataset.with_columns(
        ((pl.col("foto_mes") // 100) * 12 + (pl.col("foto_mes") % 100)).alias("periodo0")
    )

    # === 3. Calcular leads (mes siguiente y subsiguiente) por cliente ===
    dsimple = (
        dataset
        .sort(["numero_de_cliente", "periodo0"])
        .with_columns([
            pl.col("periodo0").shift(-1).over("numero_de_cliente").alias("periodo1"),
            pl.col("periodo0").shift(-2).over("numero_de_cliente").alias("periodo2"),
        ])
    )

    # === 4. Calcular topes ===
    periodo_ultimo = dsimple["periodo0"].max()
    periodo_anteultimo = periodo_ultimo - 1

    # === 5. Crear columna clase_ternaria ===
    #   Regla base: CONTINUA
    #   Luego se sobrescriben BAJA+1 y BAJA+2

    dsimple = dsimple.with_columns(pl.lit(None).alias("clase_ternaria"))

    dsimple = dsimple.with_columns(
        pl.when(pl.col("periodo0") < periodo_anteultimo)
        .then(pl.lit("CONTINUA"))              # ← pl.lit() asegura que es un valor literal
        .otherwise(pl.col("clase_ternaria"))
        .alias("clase_ternaria")
    )

    dsimple = dsimple.with_columns(
        pl.when(
            (pl.col("periodo0") < periodo_ultimo) &
            ((pl.col("periodo1").is_null()) | (pl.col("periodo0") + 1 < pl.col("periodo1")))
        )
        .then(pl.lit("BAJA+1"))                   # ← ¡usa pl.lit()!
        .otherwise(pl.col("clase_ternaria"))
        .alias("clase_ternaria")
    )

    dsimple = dsimple.with_columns(
        pl.when(
            (pl.col("periodo0") < periodo_anteultimo) &
            (pl.col("periodo0") + 1 == pl.col("periodo1")) &
            ((pl.col("periodo2").is_null()) | (pl.col("periodo0") + 2 < pl.col("periodo2")))
        )
        .then(pl.lit("BAJA+2"))                     # ← usar pl.lit() siempre
        .otherwise(pl.col("clase_ternaria"))
        .alias("clase_ternaria")
    )

    # === 6. Reinsertar en dataset original ===
    dataset = (
        dataset.join(
            dsimple.select(["numero_de_cliente", "periodo0", "clase_ternaria"]),
            on=["numero_de_cliente", "periodo0"],
            how="left"
        )
    )
    return dataset

def generar_columna_target(dataset):
    # target = 1 si clase_ternaria ∈ ["BAJA+1", "BAJA+2"], sino 0
    dataset = dataset.with_columns([
        pl.col("clase_ternaria")
          .is_in(["BAJA+1", "BAJA+2"])
          .cast(pl.Int8)
          .alias("target"),
    
        # clase_peso = 1.0
        pl.lit(1.0).alias("clase_peso")
    ])
    return dataset

def generar_delta_lags(dataset):
    # Ordenar por cliente y mes (igual que sort_values)
    dataset = dataset.sort(["numero_de_cliente", "foto_mes"])
    
    exclude_cols = {"numero_de_cliente", "foto_mes", "clase_ternaria", "target", "clase_peso"}
    
    numeric_types = (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64
    )
    
    fe_cols = [
        c for c, dtype in zip(dataset.columns, dataset.dtypes)
        if c not in exclude_cols and isinstance(dtype, numeric_types)
    ]
    
    #print(f"Columnas numéricas seleccionadas: {len(fe_cols)}")
    
    # Paso 1: crear los lagN
    for n in [1, 2]:
        lag_exprs = [
            pl.col(c).shift(n).over("numero_de_cliente").alias(f"{c}_lag{n}")
            for c in fe_cols
        ]
        dataset = dataset.with_columns(lag_exprs)
    
    # Paso 2: crear los dlagN (una vez que ya existen los lagN)
    for n in [1, 2]:
        dlag_exprs = [
            (pl.col(c) - pl.col(f"{c}_lag{n}")).alias(f"{c}_dlag{n}")
            for c in fe_cols
        ]
        dataset = dataset.with_columns(dlag_exprs)
    
    print(f"Lags/deltas agregados: {len(fe_cols)*2}")
    return dataset

def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

def undersampling_experimento(dataset,undersampling,train_months,seed):
    # --- parámetros de entrenamiento ---  
    dataset = dataset.filter(pl.col("foto_mes").is_in(train_months))

    # --- generar la columna azar reproducible ---
    rng = np.random.default_rng(seed)
    dataset = dataset.with_columns(
        pl.Series("azar", rng.random(len(dataset)))  # uniforme entre 0 y 1
    )
    
    # --- inicializar columna training ---
    dataset = dataset.with_columns(pl.lit(0).alias("training"))
    
    # --- aplicar la misma condición que en R ---
    dataset = dataset.with_columns(
        pl.when(
                (pl.col("azar") <= undersampling) |
                (pl.col("clase_ternaria").is_in(["BAJA+1", "BAJA+2"]))
        )
        .then(1)
        .otherwise(0)
        .alias("training")
    )
    
    # --- chequeo ---
    print(dataset.select(pl.col("training")).to_series().value_counts())
    print(dataset.group_by(["training", "clase_ternaria"]).len().sort("training"))
    return dataset

def convertirpercentil(dataset):
    # --- 1️⃣ Ordenar por fecha y cliente
    dataset = dataset.sort(["foto_mes", "numero_de_cliente"])
    
    # --- 2️⃣ Detectar columnas numéricas (excepto las de control)
    exclude_cols = {"numero_de_cliente", "foto_mes", "clase_ternaria", "target", "clase_peso"}
    
    numeric_types = (
        pl.Int8, pl.Int16, pl.Int32, pl.Int64,
        pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64,
        pl.Float32, pl.Float64
    )
    
    num_cols = [
        c for c, dtype in dataset.schema.items()
        if c not in exclude_cols and dtype in numeric_types
    ]
    
    #print(f"Columnas numéricas detectadas: {num_cols}")
    
    # --- 3️⃣ Función auxiliar: percentil dentro del grupo de foto_mes
    def to_percentile_expr(col):
        # rank() / count() * 100 → percentil [0,100]
        return (pl.col(col).rank("average") / pl.count() * 100).cast(pl.Float32)
    
    # --- 4️⃣ Aplicar transformación por grupo de foto_mes
    for c in num_cols:
        new_col = f"{c}_pctl"
        dataset = (
            dataset.with_columns(
                to_percentile_expr(c).over("foto_mes").alias(new_col)
            )
            .drop(c)  # quitar la original; si querés mantenerla, eliminá esta línea
        )
        #print(f"✅ Convertida {c} → {new_col}")
    
    print(f"\nTotal columnas convertidas: {len(num_cols)}")
    return dataset

In [23]:
#La primera vez ejecuten esste y despues comenten
path = "../datasets/competencia_02_crudo.csv.gz"
dataset = read_polar(path)
dataset = generar_ternaria(dataset)
dataset = generar_columna_target(dataset)
dataset = generar_delta_lags(dataset)
dataset = convertirpercentil(dataset)
#path_target = "../datasets/competencia_02.csv.gz"
#write_polar_gz(dataset,path_target)

resumen = (
    dataset
    .group_by(["foto_mes", "clase_ternaria"])
    .agg(pl.count().alias("cantidad"))
    .sort(["foto_mes", "clase_ternaria"])
)

pivot = resumen.pivot(
    values="cantidad",
    index="foto_mes",
    columns="clase_ternaria"
).fill_null(0)

print(pivot)
#pivot.write_csv("output.csv")

Lags/deltas agregados: 302
Columnas numéricas detectadas: ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad', 'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen', 'mpasivos_margen', 'cproductos', 'tcuentas', 'ccuenta_corriente', 'mcuenta_corriente_adicional', 'mcuenta_corriente', 'ccaja_ahorro', 'mcaja_ahorro', 'mcaja_ahorro_adicional', 'mcaja_ahorro_dolares', 'cdescubierto_preacordado', 'mcuentas_saldo', 'ctarjeta_debito', 'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa', 'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo', 'ctarjeta_master', 'ctarjeta_master_transacciones', 'mtarjeta_master_consumo', 'cprestamos_personales', 'mprestamos_personales', 'cprestamos_prendarios', 'mprestamos_prendarios', 'cprestamos_hipotecarios', 'mprestamos_hipotecarios', 'cplazo_fijo', 'mplazo_fijo_dolares', 'mplazo_fijo_pesos', 'cinversion1', 'minversion1_pesos', 'minversion1_dolares', 'cinversion2', 'minversion2', 'cseguro_vida'

/tmp/ipykernel_2895/3524049700.py:200: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  return (pl.col(col).rank("average") / pl.count() * 100).cast(pl.Float32)


✅ Convertida cliente_vip → cliente_vip_pctl
✅ Convertida internet → internet_pctl
✅ Convertida cliente_edad → cliente_edad_pctl
✅ Convertida cliente_antiguedad → cliente_antiguedad_pctl
✅ Convertida mrentabilidad → mrentabilidad_pctl
✅ Convertida mrentabilidad_annual → mrentabilidad_annual_pctl
✅ Convertida mcomisiones → mcomisiones_pctl
✅ Convertida mactivos_margen → mactivos_margen_pctl
✅ Convertida mpasivos_margen → mpasivos_margen_pctl
✅ Convertida cproductos → cproductos_pctl
✅ Convertida tcuentas → tcuentas_pctl
✅ Convertida ccuenta_corriente → ccuenta_corriente_pctl
✅ Convertida mcuenta_corriente_adicional → mcuenta_corriente_adicional_pctl
✅ Convertida mcuenta_corriente → mcuenta_corriente_pctl
✅ Convertida ccaja_ahorro → ccaja_ahorro_pctl
✅ Convertida mcaja_ahorro → mcaja_ahorro_pctl
✅ Convertida mcaja_ahorro_adicional → mcaja_ahorro_adicional_pctl
✅ Convertida mcaja_ahorro_dolares → mcaja_ahorro_dolares_pctl
✅ Convertida cdescubierto_preacordado → cdescubierto_preacordado_pct

/tmp/ipykernel_2895/3966937955.py:14: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("cantidad"))
/tmp/ipykernel_2895/3966937955.py:18: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  pivot = resumen.pivot(


In [24]:
train_months = [202101, 202102, 202103,202104,202105]   # meses del conjunto de entrenamiento
undersampling = 0.1                       # proporción de CONTINUA a conservar
seed = 100003                             # semilla reproducible (igual que PARAM$semilla_primigenia)
dataset = undersampling_experimento(dataset,undersampling,train_months,seed)



shape: (2, 2)
┌──────────┬────────┐
│ training ┆ count  │
│ ---      ┆ ---    │
│ i32      ┆ u32    │
╞══════════╪════════╡
│ 0        ┆ 724322 │
│ 1        ┆ 89804  │
└──────────┴────────┘
shape: (4, 3)
┌──────────┬────────────────┬────────┐
│ training ┆ clase_ternaria ┆ len    │
│ ---      ┆ ---            ┆ ---    │
│ i32      ┆ str            ┆ u32    │
╞══════════╪════════════════╪════════╡
│ 0        ┆ CONTINUA       ┆ 724322 │
│ 1        ┆ BAJA+1         ┆ 4581   │
│ 1        ┆ BAJA+2         ┆ 4780   │
│ 1        ┆ CONTINUA       ┆ 80443  │
└──────────┴────────────────┴────────┘
